In [1]:
import cv2  #OpenCV, thư viện xử lý ảnh mạnh mẽ, dùng để đọc, hiển thị và tiền xử lý ảnh.
import numpy as np    #Trong YOLO và Albumentations, NumPy là nền tảng để xử lý pixel ảnh và tọa độ bounding box.
import albumentations as A    #Dùng để tăng cường dữ liệu huấn luyện (data augmentation) mà vẫn giữ nhãn (bbox) đồng bộ.
import yaml   #Xử lý file YAML cho cấu hình dataset và các tham số huấn luyện.
from tqdm import tqdm   #Hiển thị tiến độ train
from ultralytics import YOLO    #thư viện triển khai YOLO
import torch    #YOLO sử dụng PyTorch để huấn luyện và tính toán trên GPU.
import pandas as pd

d:\newtask\LEARN_ML\.venv\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


FlashAttention is not available on this device. Using scaled_dot_product_attention instead.


In [9]:
#reload lại cache GPU
torch.cuda.empty_cache()

In [11]:
#Kiểm tra GPU
print("CUDA available:", torch.cuda.is_available())
if torch.cuda.is_available():
    print("GPU:", torch.cuda.get_device_name(0))
#Đọc file yaml cấu hình các class trong dataset và setting cho YOLO
DATA_YAML = "D:/newtask/LEARN_ML/TRAFFIC_SIGNS/dataset/dataset.yaml" 
MODEL = "yolov13n.pt"              
EPOCHS = 200
IMG_SIZE = 640
BATCH = 16
PROJECT = "runs_yolo"
NAME = "yolov13_custom_train2"

CUDA available: True
GPU: NVIDIA GeForce RTX 3050 Laptop GPU


In [2]:
DATA_YAML = "D:/newtask/LEARN_ML/TRAFFIC_SIGNS/dataset/dataset.yaml" 
MODEL = "yolov13n.pt"              
EPOCHS = 200
IMG_SIZE = 640
BATCH = 16
PROJECT = "runs_yolo"
NAME = "yolov13_custom_train2"

In [ ]:
# Huấn luyện
model = YOLO(MODEL)
results = model.train(
    data=DATA_YAML,
    imgsz=IMG_SIZE,
    epochs=EPOCHS,
    batch=BATCH,
    device=0 if torch.cuda.is_available() else 'cpu',
    project=PROJECT,
    name=NAME,
    pretrained=True,
    workers=4,
    patience=40,
    cache = True,
    degrees=7,
    translate=0.1,
    scale=0.25,
    shear=2.0,
    perspective=0.0005,
    flipud=0.0,
    fliplr=0.2,
    mosaic=0.4,
    mixup=0.05,
    copy_paste=0.0,
)

In [ ]:
#nếu dừng khi đang train, cell này dùng để train tiếp từ checkpoint
# Đường dẫn tới model checkpoint 
RESUME_MODEL = "D:/newtask/LEARN_ML/TRAFFIC_SIGNS/runs_yolo/yolov13_custom_train2/weights/last.pt"

# Tiếp tục train
model = YOLO(RESUME_MODEL)
model.train(
    resume=True,   # tiếp tục từ checkpoint
)

In [ ]:
#đánh giá mô hình
model = YOLO("D:/newtask/LEARN_ML/TRAFFIC_SIGNS/runs_yolo/yolov13_custom_train2/weights/best.pt")  
metrics = model.val(data="D:/newtask/LEARN_ML/TRAFFIC_SIGNS/dataset/dataset.yaml", imgsz=640, conf=0.001, iou=0.6)
#conf cao thì mô hình chọn lọc độ chính xác, thấp thì mô hình chọn bắt hết
#IOU cao là chỉ giữ box tin cậy (độ chồng lấp) có thể bỏ sót, thấp nhận nhiều nhưng tin cậy thấp
summary = {
    "mAP50": round(metrics.box.map50, 3),
    "mAP50-95": round(metrics.box.map, 3),
    "Precision": round(metrics.box.mp, 3),
    "Recall": round(metrics.box.mr, 3),
    "Speed (ms/img)": round(metrics.speed["inference"], 1),
}

df = pd.DataFrame([summary])
print("\n📊 Kết quả đánh giá tổng quát:")
print(df.to_markdown(index=False))

Ultralytics 8.3.63  Python-3.13.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
YOLOv13n summary: 535 layers, 2,456,280 parameters, 0 gradients, 6.2 GFLOPs


val: Scanning D:\newtask\LEARN_ML\TRAFFIC_SIGNS\dataset\val\labels.cache... 72 images, 0 backgrounds, 0 corrupt: 100%|██████████| 72/72 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50      mAP75  mAP50-95): 100%|██████████| 5/5 [00:06<00:00,  1.32s/it]


                   all         72        117      0.876      0.864      0.962      0.958      0.852
        Cm dng v           4          4      0.892          1      0.995      0.995      0.865
       Cm ngi i b          3          3      0.705          1       0.83       0.83      0.732
                Cm           8          8          1      0.941      0.995      0.995      0.885
               i chm          2          2      0.748          1      0.995      0.995      0.921
Giao nhau vi ng khng u tin         10         10      0.864        0.9      0.886      0.777       0.73
Giao vi ng u tin          3          3      0.895          1      0.995      0.995      0.931
             Hiu lnh         10         10      0.904      0.942      0.905      0.905      0.815
    Cm i ngc chiu          7          7      0.946          1      0.995      0.995      0.895
Cnh bo c ngi i b          9          9      0.953      0.889      0.956      0.956      0.776
    Cnh bo c tr em          3 

In [ ]:
#đánh giá mô hình trên tập test
model = YOLO("D:/newtask/LEARN_ML/TRAFFIC_SIGNS/runs_yolo/yolov13_custom_train2/weights/best.pt")  
metrics = model.val(data="D:/newtask/LEARN_ML/TRAFFIC_SIGNS/dataset/dataset.yaml",split="test", imgsz=640, conf=0.001, iou=0.6)
summary = {
    "mAP50": round(metrics.box.map50, 3),
    "mAP50-95": round(metrics.box.map, 3),
    "Precision": round(metrics.box.mp, 3),
    "Recall": round(metrics.box.mr, 3),
    "Speed (ms/img)": round(metrics.speed["inference"], 1),
}

df = pd.DataFrame([summary])
print("\n📊 Kết quả đánh giá tổng quát:")
print(df.to_markdown(index=False))

Ultralytics 8.3.63  Python-3.13.9 torch-2.6.0+cu124 CUDA:0 (NVIDIA GeForce RTX 3050 Laptop GPU, 4096MiB)
YOLOv13n summary: 535 layers, 2,456,280 parameters, 0 gradients, 6.2 GFLOPs


val: Scanning D:\newtask\LEARN_ML\TRAFFIC_SIGNS\dataset\test\labels... 72 images, 0 backgrounds, 0 corrupt: 100%|██████████| 72/72 [00:00<00:00, 189.56it/s]

val: New cache created: D:\newtask\LEARN_ML\TRAFFIC_SIGNS\dataset\test\labels.cache



                 Class     Images  Instances      Box(P          R      mAP50      mAP75  mAP50-95): 100%|██████████| 5/5 [00:02<00:00,  2.43it/s]


                   all         72        101      0.826      0.812      0.927      0.908      0.802
        Cm dng v           2          2      0.767          1      0.995      0.995      0.895
       Cm ngi i b          1          1          1          0      0.995      0.995      0.895
                Cm          10         10      0.827      0.959      0.898      0.898      0.827
               i chm          3          3      0.836          1      0.995      0.995      0.895
Giao nhau vi ng khng u tin          8          8      0.852      0.875      0.944      0.944      0.761
Giao vi ng u tin          2          2      0.803        0.5      0.638      0.516      0.477
             Hiu lnh          7          7      0.911          1      0.995      0.995      0.875
    Cm i ngc chiu          5          5      0.937          1      0.995      0.995      0.861
Cnh bo c ngi i b          3          3      0.894          1      0.995      0.995      0.842
    Cnh bo c tr em          3 

In [10]:
#dự đoán trên ảnh đưa vào máy
image_path = "D:/newtask/LEARN_ML/12.jpg"
model = YOLO('D:/newtask/LEARN_ML/TRAFFIC_SIGNS/runs_yolo/yolov13_custom_train2/weights/last.pt')
results = model.predict(source=image_path, conf=0.25, iou=0.45, save=True)


image 1/1 D:\newtask\LEARN_ML\12.jpg: 480x640 1 Cm dng v , 1 Cm , 1 Giao nhau vi ng khng u tin, 1 Cm i ngc chiu, 2 Cm r phis, 3 Cm oto r tris, 1 Cm quay u, 1 ng nguy him vng bn tri, 1 Giao vi ng cng cp, 1 Gii hn tc  50km/h, 1 Gii hn tc  80km/h, 1 Cm xe xch l, 92.5ms
Speed: 5.2ms preprocess, 92.5ms inference, 507.4ms postprocess per image at shape (1, 3, 480, 640)
Results saved to runs\detect\predict7


In [2]:
model = YOLO("D:/newtask/LEARN_ML/TRAFFIC_SIGNS/runs_yolo/yolov13_custom_train2/weights/best.pt")

#  Mở webcam (0 = mặc định)
cap = cv2.VideoCapture(0)

if not cap.isOpened():
    print("Không thể mở webcam!")
    exit()

#  Vòng lặp nhận diện từng frame
while True:
    ret, frame = cap.read()
    if not ret:
        print("Không đọc được khung hình.")
        break

    # Nhận diện trực tiếp
    results = model.predict(frame, imgsz=640, conf=0.25, device=0, verbose=False)

    # Vẽ bounding box lên frame (hàm render của ultralytics)
    annotated_frame = results[0].plot()

    # Hiển thị
    cv2.imshow("YOLOv13 - Traffic Sign Detection", annotated_frame)

    # Nhấn 'q' để thoát
    if cv2.waitKey(1) & 0xFF == ord('q'):
        break

#  Giải phóng tài nguyên
cap.release()
cv2.destroyAllWindows()